# $SO(3)$ and $so(3)$
Given two rotation matrices $R_i$ and $R_j$ find the relative rotation $R_{ij}$ solving
\begin{align*}
    \min \lVert R_j - R_{ij} R_i \rVert^2
\end{align*}
using the $so(3)$ reparametrization
\begin{align*}
    \min_{w \in \mathbb{R}^3} \lVert R_j - \exp(w) R_i \rVert^2
\end{align*}
The correct solution is given by
\begin{align*}
    R_{ij} = R_j R_i^{-1} 
\end{align*}


In [85]:
import numpy as np
import scipy.optimize as optimize
from scipy.spatial.transform import Rotation

In [86]:
def skew(w):
    return np.array([[0, -w[2], w[1]], 
                     [w[2], 0, -w[0]], 
                     [-w[1], w[0], 0]])
def rodrigues(w):
    w_mod = np.linalg.norm(w)
    if w_mod == 0:
        return np.eye(3)

    w_hat = skew(w)
    w_norm = w_hat / w_mod
    return np.eye(3) + w_norm * np.sin(w_mod) + w_norm @ w_norm * (1 - np.cos(w_mod))

In [87]:
def loss(x, R_i, R_j):
    R_ij = rodrigues(x)
    return np.linalg.norm(R_j - R_ij @ R_i)**2

In [88]:
# Generate a source and a target rotations
R_i = Rotation.random()
R_j = Rotation.random()

# Solution is R_j (R_i)^-1
R_ij_gt = R_j.as_matrix() @ R_i.inv().as_matrix() 

# Optimize
res = optimize.least_squares(
    fun=loss, 
    x0=np.zeros(3),
    loss='linear',
    xtol=1e-12,
    ftol=1e-12,
    gtol=1e-12,
    verbose=2,
    kwargs = {
        'R_i': R_i.as_matrix(),
        'R_j': R_j.as_matrix()
    }
)

# Get solution
R_ij = rodrigues(res.x)
print("Loss:", np.linalg.norm(R_ij_gt - R_ij)**2)



   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.7388e+01                                    1.57e+01    
       1              2         2.1256e+00      1.53e+01       1.00e+00       5.46e+00    
       2              3         1.3201e+00      8.06e-01       2.00e+00       3.96e+00    
       3              4         6.9408e-02      1.25e+00       5.00e-01       4.75e-01    
       4              6         2.3312e-03      6.71e-02       2.50e-01       3.80e-02    
       5              8         2.5967e-05      2.31e-03       1.25e-01       1.31e-03    
       6             10         7.3706e-11      2.60e-05       6.25e-02       9.05e-08    
       7             12         8.6565e-12      6.51e-11       3.91e-03       1.82e-08    
       8             13         9.4160e-14      8.56e-12       9.77e-04       6.12e-10    
       9             15         5.0992e-19      9.42e-14       4.88e-04       6.86e-14    